In [ ]:
def paren(n, edges):
    """
    Given the number of nodes (n) and a list of edges (each an (u, v) pair) 
    representing an undirected tree, returns a canonical parenthesis (bracket) 
    representation of that tree.

    Example:
        n = 5
        edges = [(0,1), (0,2), (2,3), (2,4)]
        The function returns a string like "((()())())" (the exact shape 
        depends on how children subtrees get sorted).
    """

    # Build adjacency list for the tree
    adj = [[] for _ in range(n)]
    for u, v in edges:
        adj[u].append(v)
        adj[v].append(u)

    # Recursive helper function that returns the bracket representation 
    # of the subtree rooted at `node`, with `parent` as its parent.
    def dfs(node, parent):
        # Collect child representations
        child_reps = []
        for nxt in adj[node]:
            if nxt != parent:
                child_reps.append(dfs(nxt, node))

        # Sort the children’s bracket strings to get a canonical form
        child_reps.sort()
        # Combine them and enclose in parentheses
        return "(" + "".join(child_reps) + ")"

    # Run DFS from any node as root (we pick 0, assuming the tree is connected)
    # If your tree is not guaranteed to be connected, 
    # you would first check connectivity or pick a valid root in another way.
    return dfs(0, -1)
def paren_to_tree(parens):
    """
    Reconstructs a tree adjacency list from a canonical parentheses representation.

    Args:
        parens (str): A valid parentheses sequence representing a tree. 
                      E.g. '((()())())'

    Returns:
        edges (list of (int,int)): A list of undirected edges (u,v)
        or
        adj (list of lists): An adjacency list for the corresponding tree.

    Notes:
        - Each "()" pair corresponds to exactly one node.
        - The total number of pairs of parentheses is the total number of nodes.
        - If the string has k pairs of parentheses, that means we have k nodes
          labeled from 0 to k-1.

    Example:
        paren_seq = "((()())())"
        edges = paren_to_tree(paren_seq)
        # edges might look like:
        # [(0, 1), (1, 2), (1, 3), (0, 4)]
    """
    stack = []
    edges = []
    next_id = 0   # label nodes from 0, 1, 2, ...

    # We'll keep an adjacency list in case you want adjacency at the end
    # The maximum number of nodes = length(parens)//2 
    # (since every node is represented by one pair of parentheses).
    n = len(parens) // 2
    adj = [[] for _ in range(n)]

    for char in parens:
        if char == '(':
            # Create a new node
            node_id = next_id
            next_id += 1

            # If there is a parent (the top of the stack), connect to it
            if stack:
                parent = stack[-1]
                edges.append((parent, node_id))
                # If you also want adjacency:
                adj[parent].append(node_id)
                adj[node_id].append(parent)

            # Push the new node onto the stack
            stack.append(node_id)

        elif char == ')':
            # Done with the current node, pop from stack
            stack.pop()
    
    return edges, n  # or return adj if you want adjacency instead

from itertools import permutations
paren_to_tree(paren(5, [(0, 1), (0, 2), (1, 3), (1, 4)]))
def are_eq(p1, p2):
    n = len(p1)
    t1, n = paren_to_tree(p1)
    t2, n = paren_to_tree(p2)
    for p in permutations([i for i in range(n)]):
        new_t = t1.copy()
        for i in range(len(new_t)):
            new_t[i] = p[new_t[i][0]], p[new_t[i][1]]
        new_p = paren(n, new_t)
        if new_p == p2:
            return True
    return False
from tqdm import *
N = 9
trees = [[] for _ in range(N)]
trees[3].append(paren(3, [(0, 1), (1, 2)]))
for cnt in tqdm(range(4, N)):
    for old_tree in trees[cnt - 1]:
        # Add new edge to random node
        t, n = paren_to_tree(old_tree)
        temp_n = n + 1
        for q in range(n):
            temp_t = t.copy()
            temp_t.append((n, q))
            # Check if eq to any ones already in cnt
            parens = paren(temp_n, temp_t)
            ins = True
            for other in trees[cnt]:
                if are_eq(parens, other):
                    ins = False
                    break
            if ins:
                print("Added")
                trees[cnt].append(parens)


print([len(e) for e in trees])


In [65]:
def find_centroids(n, adj):
    deg = [len(adj[u]) for u in range(n)]
    leaves = [u for u in range(n) if deg[u] <= 1]
    remaining = n
    while remaining > 2:
        new_leaves = []
        remaining -= len(leaves)
        for leaf in leaves:
            deg[leaf] = 0
            for nei in adj[leaf]:
                if deg[nei] > 0:
                    deg[nei] -= 1
                    if deg[nei] == 1:
                        new_leaves.append(nei)
        leaves = new_leaves
    return leaves

def canonical_dfs(u, parent, adj):
    forms = []
    for v in adj[u]:
        if v != parent:
            forms.append(canonical_dfs(v, u, adj))
    forms.sort()
    return "(" + "".join(forms) + ")"

def canonical_form(n, edges):
    # Build adjacency
    adj = [[] for _ in range(n)]
    for (a, b) in edges:
        adj[a].append(b)
        adj[b].append(a)

    # Find centroid(s) and build the canonical parentheses form
    cents = find_centroids(n, adj)
    candidates = []
    for c in cents:
        candidates.append(canonical_dfs(c, -1, adj))
    return min(candidates)

def paren_to_tree(parens):
    """
    Converts a parenthesis representation into an edge list and number of nodes.
    """
    stack = []
    edges = []
    next_id = 0
    n = len(parens)//2  # number of nodes
    for ch in parens:
        if ch == '(':
            node = next_id
            next_id += 1
            if stack:
                parent = stack[-1]
                edges.append((parent, node))
            stack.append(node)
        elif ch == ')':
            stack.pop()
    return edges, n

def generate_unlabeled_trees_up_to(N):
    """
    Returns a list 'trees' of length N+1, where trees[n] is a list of
    canonical forms (strings) for all unlabeled trees on n nodes.
    """
    from tqdm import tqdm

    trees = [[] for _ in range(N+1)]
    if N < 3:
        return trees

    # Base case for n=3: the only (unlabeled) tree is the path 0--1--2
    base_canon = canonical_form(3, [(0,1), (1,2)])
    trees[3] = [base_canon]

    for cnt in range(4, N+1):
        print(f"Generating size={cnt} trees...")
        seen = set()
        for old_canon in trees[cnt - 1]:
            # Convert old canonical form back to edges
            old_edges, old_size = paren_to_tree(old_canon)

            # Attach a new node 'old_size' to each existing node in [0..old_size-1]
            for attach_point in range(old_size):
                new_edges = old_edges + [(old_size, attach_point)]
                # Get canonical form of the resulting bigger tree
                new_canon = canonical_form(cnt, new_edges)
                seen.add(new_canon)
        trees[cnt] = list(seen)

    return trees

if __name__ == "__main__":
    N = 16
    all_trees = generate_unlabeled_trees_up_to(N)

    # Now filter out any trees that have an edge with same-degree endpoints.
    total_valid = 0

    tv = []
    for n in range(3, N+1):
        count_for_n = 0
        for cform in all_trees[n]:
            # Convert parentheses -> edges
            edges, _ = paren_to_tree(cform)

            # Compute degrees
            deg = [0]*n
            for (u, v) in edges:
                deg[u] += 1
                deg[v] += 1

            # Check each edge for deg[u] == deg[v]
            invalid = False
            for (u, v) in edges:
                if deg[u] == deg[v]:
                    invalid = True
                    break

            if not invalid:
                count_for_n += 1

        print(f"n={n}: number of valid trees =", count_for_n)
        tv.append(str(count_for_n))
        total_valid += count_for_n

    print(",".join(tv))

    print("Overall valid trees across all n:", total_valid)


Generating size=4 trees...
Generating size=5 trees...
Generating size=6 trees...
Generating size=7 trees...
Generating size=8 trees...
Generating size=9 trees...
Generating size=10 trees...
Generating size=11 trees...
Generating size=12 trees...
Generating size=13 trees...
Generating size=14 trees...
Generating size=15 trees...
Generating size=16 trees...
n=3: number of valid trees = 1
n=4: number of valid trees = 1
n=5: number of valid trees = 2
n=6: number of valid trees = 3
n=7: number of valid trees = 6
n=8: number of valid trees = 9
n=9: number of valid trees = 19
n=10: number of valid trees = 33
n=11: number of valid trees = 67
n=12: number of valid trees = 130
n=13: number of valid trees = 270
n=14: number of valid trees = 547
n=15: number of valid trees = 1165
n=16: number of valid trees = 2456
1,1,2,3,6,9,19,33,67,130,270,547,1165,2456
Overall valid trees across all n: 4709


In [68]:
tv = [int(e) for e in tv]
tv

[1, 1, 2, 3, 6, 9, 19, 33, 67, 130, 270, 547, 1165, 2456]

In [76]:
for i in range(1, len(tv)):
    print(tv[i] / tv[i - 1], tv[i - 1], tv[i - 1], int(tv[i - 1] * 2.1), tv[i])

1.0 1 1 2 1
2.0 1 1 2 2
1.5 2 2 4 3
2.0 3 3 6 6
1.5 6 6 12 9
2.111111111111111 9 9 18 19
1.736842105263158 19 19 39 33
2.0303030303030303 33 33 69 67
1.9402985074626866 67 67 140 130
2.076923076923077 130 130 273 270
2.025925925925926 270 270 567 547
2.129798903107861 547 547 1148 1165
2.108154506437768 1165 1165 2446 2456


In [79]:
def find_centroids(n, adj):
    deg = [len(adj[u]) for u in range(n)]
    leaves = [u for u in range(n) if deg[u] <= 1]
    remaining = n
    while remaining > 2:
        new_leaves = []
        remaining -= len(leaves)
        for leaf in leaves:
            deg[leaf] = 0
            for nei in adj[leaf]:
                if deg[nei] > 0:
                    deg[nei] -= 1
                    if deg[nei] == 1:
                        new_leaves.append(nei)
        leaves = new_leaves
    return leaves

def canonical_dfs(u, parent, adj):
    forms = []
    for v in adj[u]:
        if v != parent:
            forms.append(canonical_dfs(v, u, adj))
    forms.sort()
    return "(" + "".join(forms) + ")"

def canonical_form(n, edges):
    """Compute AHU-based canonical form for an unlabeled tree with `n` nodes."""
    adj = [[] for _ in range(n)]
    for (a, b) in edges:
        adj[a].append(b)
        adj[b].append(a)
    cents = find_centroids(n, adj)
    candidates = []
    for c in cents:
        candidates.append(canonical_dfs(c, -1, adj))
    return min(candidates)

def paren_to_tree(parens):
    """Convert a parentheses representation to an edge list (and number of nodes)."""
    stack = []
    edges = []
    next_id = 0
    n = len(parens)//2
    for ch in parens:
        if ch == '(':
            node = next_id
            next_id += 1
            if stack:
                parent = stack[-1]
                edges.append((parent, node))
            stack.append(node)
        elif ch == ')':
            stack.pop()
    return edges, n


def generate_trees_up_to(N):
    """
    Generates all unlabeled trees (up to N nodes) that do NOT have an
    edge connecting two nodes of the same degree.

    Returns: a list 'trees' of length N+1, where trees[n] is a list of
             canonical forms for valid trees on n nodes.
    """
    # We'll store (edges, deg_array) in some steps, or just store canonical forms.
    # Then from the canonical form we can rebuild edges. 
    # But we also need a quick way to read off degrees. Let's store adjacency too.

    # For each n, we'll keep a list of (canonical_str, edges, deg_list).
    # Or we can store adjacency in a canonical order. But let's keep it simpler:
    from collections import defaultdict
    
    trees = [[] for _ in range(N+1)]
    
    # base n=3: only 1 tree, the path (0-1-2).
    # let's define it explicitly
    if N < 3:
        return trees
    
    base_edges = [(0,1), (1,2)]
    base_deg = [1, 2, 1]  # node0:deg1, node1:deg2, node2:deg1
    base_canon = canonical_form(3, base_edges)
    trees[3] = [(base_canon, base_edges, base_deg)]

    # For n=4..N:
    for n in range(4, N+1):
        seen = {}
        for (old_canon, old_edges, old_deg) in trees[n-1]:
            # old_size = n-1
            # For each old vertex u, attempt to attach new node = (n-1)
            # Check degrees to avoid adjacency with same-degree endpoints
            for u in range(n-1):
                # old_deg[u] changes to old_deg[u] + 1
                # The new node's degree is 1
                # We must check if old_deg[u]+1 == old_deg[x] for any neighbor x of u
                # So we need adjacency for the old tree:
                # build adjacency quickly for old tree
                adj_old = [[] for _ in range(n-1)]
                for (a,b) in old_edges:
                    adj_old[a].append(b)
                    adj_old[b].append(a)
                # check neighbors of u
                conflict = False
                new_deg_u = old_deg[u] + 1
                for v in adj_old[u]:
                    if old_deg[v] == new_deg_u:
                        # conflict
                        conflict = True
                        break

                # Also check if the new node of deg=1 is connecting to
                # a node of deg=1 => conflict. 
                # Actually, you only need to check if new_deg_u == 1 => that means old_deg[u] = 0,
                # which can't happen in a connected tree. But let's be thorough:
                if new_deg_u == 1:
                    # That would mean old_deg[u]==0 => not possible for connected tree
                    conflict = True

                if conflict:
                    continue  # skip this attachment

                # Ok, no conflict. Build new edges, new deg, compute canonical form
                new_edges = old_edges + [(u, n-1)]
                new_deg = old_deg[:] + [1]  # copy + new node
                new_deg[u] = new_deg_u

                # compute canonical form
                new_canon = canonical_form(n, new_edges)
                if new_canon not in seen:
                    seen[new_canon] = (new_edges, new_deg)
        
        # finalize list of trees for size n
        trees[n] = [(k, seen[k][0], seen[k][1]) for k in seen]

    return trees


if __name__ == "__main__":
    N = 20
    all_trees = generate_trees_up_to(N)

    # Let's just print how many valid trees for n=3..N
    for n in range(3, N+1):
        print(f"n = {n}, number of valid trees = {len(all_trees[n])}")


n = 3, number of valid trees = 1
n = 4, number of valid trees = 1
n = 5, number of valid trees = 2
n = 6, number of valid trees = 3
n = 7, number of valid trees = 5
n = 8, number of valid trees = 7
n = 9, number of valid trees = 13
n = 10, number of valid trees = 20
n = 11, number of valid trees = 34
n = 12, number of valid trees = 55
n = 13, number of valid trees = 93
n = 14, number of valid trees = 154
n = 15, number of valid trees = 264
n = 16, number of valid trees = 445
n = 17, number of valid trees = 763
n = 18, number of valid trees = 1303
n = 19, number of valid trees = 2240
n = 20, number of valid trees = 3846
